## 🆕 Requesting Portfolio Data###

- **Initial Approach:**
  
  * Started off by using ```MultiIndex()``` as a way to group stocks and options
  * The point of using ```MultiIndex()``` was to help in determining total equities value and total options value and organize the data
  * I tried to do everything in one dataframe but I didn't get the results the way I wanted
  
- **Improved Approach:**
  
   * ✅Step 1: Create table for Total Equities Market Value , Total Options Market Value , Total Market Value, Mkt Val %
       * ```MulitIndex()``` is only useful for formating the way the dataframe is printed,I played around with it a lot but didn't really get the desired results, it is better to use ```groupby()``` and ```getgroup()``` in conjunction with ```MultiIndex()```
       * Once grouped into Stocks and Options, split the dataframe in two, one for stocks and the other for options:
         * Sum the market values of each stock / option to determine the Total Equities/Options Market Value
         * Determine percentages
         * Append results to each dataframe
      * Create dataframe for total market values
  
  * ✅❗Step 2: Add **Post Expiry Market Value** and **Post Expiry Market Value %**
      
      * ```portfolio()``` returned a dataframe which had all the required columns, they just had different names such as
         * **'averageCost'** is the same as **'Cost Price'**
         * **'unrealizedPNL'** is the same as **'P&L'**
         * **'position'** is the same as **'Qty'**
      * The columns were renamed and rearranged
      * Additional columns / rows that were added:
         * **'% Market Value'**  for stocks protfolio
         * **'Totals'** row  which displays **'Total Market Value'**, **'Total % Market Value'** and **'Total P&L'** for each portfolio
         * **'Type','Stirke', 'Underlying Equity Market Price', 'Total Market Value Post Expiry'** for options portfolio
      * ❗ For **'Underlying Equity Market Price'**:
         * I use the symbol / Ticker to create a contract
         * Qualify the contract
         * Get ticker for the contract, takes 11 seconds
         * Get market price which will be used as **'Underlying Equity Market Price'**
      * For **'Post Expiry Market Value'**:
         * I am creating creating the required conditions as shown in sample portfolio excel
         * Based on my understanding if it is a Put, post expiry market is calculated as follows:
             * *(Strike - Underlying Equity) * Quantity * 100*
         * If it is a Call,post expiry market is calculated as follows:
             * *(Underlying Equity - Strike) * Quantity * 100*
         * 🆕 ✅ If **expired**, then post expiry market value is 0
         
- **Problems:**
    * For **'Underlying Equity Market Price'**, I understand that it is the Market Price of the equity, however I am not sure if I should extract it from the stocks portfolio or create a contract, get its ticker and then the market price
    * ✅ Options have an attribute ```lastTradeDateOrContractMonth```,but when is an option expired ? Is it when the current date ( today's date) is after the ```lastTradeDateOrContractMonth``` ?
    * I understand the calculation behind **Post Expiry Market Value**, I am doing the calcualtion but how is it relevant ? For example when it comes to P&L, I understand that if you bought(market Price) at a price higher than the Cost Price, then you bought it more expensive which is represented by a positive value, on the other hand if you bought at a price lower than cost price, you actaully bought it cheaper, you made a profit which is represented by a negative value
         

In [1]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

#Import ib_insync library
from ib_insync import *

#Only used in interactive environments such as Jupyter Notebooks
util.startLoop()

#Instantiate IB class and use .connect() method on it, if TWS is not running ConnectionRefusedError will be raised
ib = IB()
try:
    print(ib.connect(clientId=0))
except:
    pass

<IB connected to 127.0.0.1:7497 clientId=0>


In [2]:
#Automatically added to portfolio in TWS, get portfolio
portfolio = ib.portfolio()
df = util.df(portfolio)

In [3]:
#Extract contracts from portfolio
column = df['contract']
market_values = df['marketValue'].values

contract_types = []
tickers = []
pc = []
strikes = []
exchanges = []
expiry = []

#Extract the ticker and the type of contract
for obj in column:
    tickers.append(obj.symbol)
    contract_types.append(obj.__class__.__name__)
    if obj.__class__.__name__ == 'Option':
        pc.append(obj.right)
        strikes.append(obj.strike)
        exchanges.append(obj.primaryExchange)
        expiry.append(datetime.strptime(obj.lastTradeDateOrContractMonth,"%Y%m%d"))

In [4]:
#Muniplate portfolio, drop uneeded columns
df = df.drop(columns=['contract','realizedPNL','account'])

In [5]:
#Rename columns
df = df.rename(columns={"position": "Qty", "marketPrice": "Market Price", "averageCost":"Cost Price" , "unrealizedPNL":"P&L", "marketValue": "Total Market Value"})

#Add Type and Ticker columns
df['Type'] = contract_types
df['Ticker'] = tickers

#Rearrange columns
df = df[["Type","Ticker","Qty", "Market Price", "Cost Price", "P&L", "Total Market Value"]]

In [6]:
#Today's date
today = datetime.now()


#Group by Type, either Stock or Option
grouped = df.groupby('Type')

#Split Dataframe into two, one dataframe for stocks, one for options
df_stocks = grouped.get_group('Stock')
df_stocks = df_stocks.drop(columns=['Type'])
df_options = grouped.get_group('Option')
df_options = df_options.drop(columns=['Type'])

#Total Market Values
total_equities_market_value = df_stocks['Total Market Value'].sum()
total_options_market_value = df_options['Total Market Value'].sum()
total_market_value = total_equities_market_value + total_options_market_value

#Determine percentage values for stock market and option market value
percent_equities =  (total_equities_market_value / total_market_value) * 100 
percent_options = (total_options_market_value / total_market_value) * 100


# Calculating Percentage
df_stocks['% Market Value'] = (df_stocks['Total Market Value'] / total_equities_market_value) * 100
df_options['% Market Value'] = (df_options['Total Market Value'] / total_options_market_value) * 100

#Determine total P&L for stocks and options
total_pl_stocks = df_stocks['P&L'].sum()
total_pl_options = df_options['P&L'].sum()
total_pl = total_pl_stocks + total_pl_options 

#Create rows for total market value and percentages
row_equities = pd.Series({'P&L':total_pl_stocks,'Total Market Value':total_equities_market_value, '% Market Value':percent_equities }, name='Total')
row_options = pd.Series({'P&L':total_pl_options,'Total Market Value':total_options_market_value, '% Market Value':percent_options}, name='Total')


#Get underlying market value for option equity
underlying_market_equitys = []
for symbol,exchange in zip(df_options['Ticker'],exchanges):
    
    #Create contract, for each symbol
    contract = Stock(symbol,exchange,'USD')
    #Qualify the contract
    ib.qualifyContracts(contract)
    #Request ticker and market price
    [ticker] = ib.reqTickers(contract)
    underlying_market_equitys.append(ticker.marketPrice())
    
#Add column for underlying market equity
df_options['Underlying Equity Market Price'] = underlying_market_equitys
    
#Create Call/Put, Strike Column, Expiry for Options
df_options['Type'] = pc
df_options['Strike'] = strikes
df_options['Expiry'] = expiry

#Determine post expiry market value, if expired option value is zero
post_expiry_market_series = [((df_options['Strike'] - df_options['Underlying Equity Market Price']) * df_options['Qty'] * 100)
                                        if (pc =='P' and today < expiry)  
                                        else 
                                        ((df_options['Underlying Equity Market Price'] - df_options['Strike'])  * df_options['Qty'] * 100) 
                                        if (pc =='C' and today < expiry)
                                        else
                                        '0'
                                        for (pc,expiry) in zip(df_options['Type'],df_options['Expiry'])]

post_expiry_market_values = []
for series in post_expiry_market_series:
    post_expiry_market_values.append(series.tolist()[0])
    
df_options['Total Market Value Post Expiry'] = post_expiry_market_values


#Append rows to stock and options portfolio
df_stocks = df_stocks.append(row_equities, ignore_index=False)
df_stocks = df_stocks.fillna('')
df_options = df_options.append(row_options, ignore_index=False)
df_options = df_options.fillna('')



#Create totals dataframe
market_values = []
pl = []
market_values.extend([total_equities_market_value,total_options_market_value, total_market_value])
pl.extend([total_pl_stocks,total_pl_options, total_pl])
list_of_tuples = list(zip(market_values, pl)) 
df_total = pd.DataFrame(data=list_of_tuples, columns=['Market Value','P&L'], index=['Total Equities', 'Total Options', 'Total'])


In [7]:
#Display results
display(df_stocks.style.set_caption("Stocks Portfolio"))
display(df_options.style.set_caption("Options Portfolio"))
display(df_total.style.set_caption("Totals Portfolio"))

,Ticker,Qty,Market Price,Cost Price,P&L,Total Market Value,% Market Value
0,AAPL,1.000000,129.589996,126.883300,2.710000,129.590000,3.902396
1,FB,1.000000,310.940002,309.188400,1.750000,310.940000,9.363463
2,QQQ,5.000000,333.589996,327.358660,31.160000,1667.950000,50.227657
3,SHOP,1.000000,1212.300049,1175.233500,37.070000,1212.300000,36.506483
Total,,,,,72.690000,3320.780000,73.841385


,Ticker,Qty,Market Price,Cost Price,P&L,Total Market Value,% Market Value,Underlying Equity Market Price,Type,Strike,Expiry,Total Market Value Post Expiry
4,SHOP,1.000000,11.764025,5441.932800,-4265.530000,1176.400000,100.000000,1209.000000,P,1160.000000,2021-04-16 00:00:00,-4900.000000
Total,,,,,-4265.530000,1176.400000,26.158615,,,,,


,Market Value,P&L
Total Equities,3320.780000,72.690000
Total Options,1176.400000,-4265.530000
Total,4497.180000,-4192.840000


In [8]:
ib.disconnect()